# 8장 노트정리

날짜 : 2025 - 10 - 08
저자 : 차일권
학번 : 202111467


# 예외처리

예외는 프로그램 실행 중에 오류가 발생하면 생성되는 특별한 객체다.

## 주요 예외 종류
- `ZeroDivisionError`: 0으로 나눌 때
- `IndexError`: 인덱스가 범위를 벗어날 때
- `FileNotFoundError`: 존재하지 않는 파일을 열려고 할 때
- `ValueError`: 원하는 값을 입력받지 못할 때
- `NameError`: 정의되지 않은 변수를 사용할 때
- `TypeError`: 데이터형이 맞지 않는 연산을 할 때


In [ ]:
# 예외 발생 예제
print(5/0)  # ZeroDivisionError
a = [0, 1, 2]
print(a[3])  # IndexError
int('abc')  # ValueError


## try-except 문

```python
try:
    # 실행할 코드
    result = 10 / 0
except ZeroDivisionError:
    print("0으로 나눌 수 없습니다")
except Exception as e:
    print(f"예외 발생: {e}")
else:
    print("예외가 발생하지 않았습니다")
finally:
    print("항상 실행됩니다")
```


In [ ]:
# 예외처리 예제
try:
    num = int(input("숫자를 입력하세요: "))
    result = 10 / num
    print(f"결과: {result}")
except ValueError:
    print("숫자가 아닙니다")
except ZeroDivisionError:
    print("0으로 나눌 수 없습니다")
except Exception as e:
    print(f"예외 발생: {e}")
finally:
    print("프로그램 종료")


## 사용자 정의 예외

```python
class NegativeNumberError(Exception):
    def __init__(self, value):
        self.value = value
        self.message = f"음수는 허용되지 않습니다: {value}"
        super().__init__(self.message)

def process_number(n):
    if n < 0:
        raise NegativeNumberError(n)
    return n * 2
```


In [ ]:
# 사용자 정의 예외 예제
class NegativeNumberError(Exception):
    def __init__(self, value):
        self.value = value
        self.message = f"음수는 허용되지 않습니다: {value}"
        super().__init__(self.message)

def process_number(n):
    if n < 0:
        raise NegativeNumberError(n)
    return n * 2

try:
    num = int(input("양의 정수를 입력하세요: "))
    result = process_number(num)
    print(f"처리 결과: {result}")
except ValueError:
    print("숫자가 아닙니다")
except NegativeNumberError as e:
    print(f"사용자 정의 예외: {e}")


# 파일 처리

## 파일 열기 모드
- `'r'`: 읽기 전용
- `'w'`: 쓰기 전용 (덮어쓰기)
- `'a'`: 추가 모드
- `'x'`: 배타적 생성 모드
- `'r+'`: 읽기/쓰기 모드
- `'w+'`: 읽기/쓰기 모드 (덮어쓰기)
- `'a+'`: 읽기/쓰기 모드 (추가)


In [ ]:
# 텍스트 파일 쓰기
with open("phonebook.txt", "w", encoding="utf-8") as f:
    f.write("홍길동, 010-1234-5678\n")
    f.write("김철수, 010-2345-6789\n")
    f.write("이영희, 010-3456-7890\n")
print("파일 쓰기 완료")


In [ ]:
# 텍스트 파일 읽기
try:
    with open("phonebook.txt", "r", encoding="utf-8") as f:
        for line in f:
            print(line.strip())
except FileNotFoundError:
    print("파일을 찾을 수 없습니다")


In [ ]:
# 딕셔너리를 파일에 저장하고 읽기
contacts = {
    '홍길동': '010-1234-5678',
    '김철수': '010-2345-6789',
    '이영희': '010-3456-7890'
}

# 쓰기
with open("contacts.txt", "w", encoding="utf-8") as f:
    for name, phone in contacts.items():
        f.write(f"{name}, {phone}\n")

# 읽기
with open("contacts.txt", "r", encoding="utf-8") as f:
    for line in f:
        name, phone = line.strip().split(", ")
        print(f"이름: {name}, 전화번호: {phone}")


## 이진 파일 처리

이진 파일은 이미지, 음악, 영상 등 텍스트가 아닌 데이터를 저장한다.


In [ ]:
import struct

# 연락처 클래스
class ContactRecord:
    def __init__(self, phone: str, name: str, age: int = 0):
        self.phone = phone
        self.name = name
        self.age = age
    
    def __str__(self):
        return f"전화번호: {self.phone}, 이름: {self.name}, 나이: {self.age}"

# 레코드 패킹/언패킹 함수
def pack_contact_record(contact: ContactRecord) -> bytes:
    phone_bytes = contact.phone.encode('utf-8')[:16].ljust(16, b'\x00')
    name_bytes = contact.name.encode('utf-8')[:16].ljust(16, b'\x00')
    return struct.pack("16s16si", phone_bytes, name_bytes, contact.age)

def unpack_contact_record(record: bytes) -> ContactRecord:
    phone_bytes, name_bytes, age = struct.unpack("16s16si", record)
    phone = phone_bytes.rstrip(b'\x00').decode('utf-8')
    name = name_bytes.rstrip(b'\x00').decode('utf-8')
    return ContactRecord(phone, name, age)

# 테스트 데이터
contacts = [
    ContactRecord("010-1234-5678", "홍길동", 25),
    ContactRecord("010-2345-6789", "김철수", 30),
    ContactRecord("010-3456-7890", "이영희", 28)
]

# 이진 파일에 쓰기
with open("contacts.bin", "wb") as f:
    for contact in contacts:
        record = pack_contact_record(contact)
        f.write(record)
        print(f"저장: {contact}")

print("\n이진 파일 읽기:")
with open("contacts.bin", "rb") as f:
    record_num = 1
    while True:
        record = f.read(36)  # 16+16+4 = 36바이트
        if not record or len(record) < 36:
            break
        contact = unpack_contact_record(record)
        print(f"레코드 {record_num}: {contact}")
        record_num += 1


## 파일의 임의 접근

`f.seek(offset, from)` 함수로 특정 위치로 이동할 수 있다.
- `from`: 0(파일 처음), 1(현재 위치), 2(파일 끝)
- `f.tell()`: 현재 위치 반환


In [ ]:
# 파일 임의 접근 예제
with open("test.txt", "w+") as f:
    f.write("Hello World!")
    f.seek(0)  # 파일 처음으로 이동
    content = f.read()
    print(f"읽은 내용: {content}")
    
    f.seek(6)  # 6번째 위치로 이동
    f.write("Python")  # "World"를 "Python"으로 변경
    
    f.seek(0)
    content = f.read()
    print(f"수정된 내용: {content}")


# F-string

F-string은 변수나 표현식을 문자열에 삽입할 수 있는 포맷 문자열이다.

## 기본 사용법
```python
name = "홍길동"
age = 25
print(f"이름: {name}, 나이: {age}")
```


In [ ]:
# F-string 기본 예제
name = "홍길동"
age = 25
score = 85.5

print(f"이름: {name}, 나이: {age}, 점수: {score}")
print(f"나이 + 10 = {age + 10}")
print(f"점수는 {'합격' if score >= 80 else '불합격'}")


## 숫자 포맷팅

- `{값:포맷}` 형태로 사용
- 소수점 자릿수: `.2f`
- 천 단위 구분: `,`
- 정렬: `<`(왼쪽), `>`(오른쪽), `^`(가운데)
- 패딩: `0`으로 채우기


In [ ]:
# 숫자 포맷팅 예제
number = 1234567.89

print(f"소수점 2자리: {number:.2f}")
print(f"천 단위 구분: {number:,}")
print(f"천 단위 구분 + 소수점: {number:,.2f}")
print(f"왼쪽 정렬: {number:<15,.2f}")
print(f"오른쪽 정렬: {number:>15,.2f}")
print(f"가운데 정렬: {number:^15,.2f}")
print(f"0으로 패딩: {number:015.2f}")

# 진법 변환
num = 255
print(f"10진수: {num}")
print(f"2진수: {num:b}")
print(f"8진수: {num:o}")
print(f"16진수: {num:x}")
print(f"16진수(대문자): {num:X}")


## 날짜와 시간 포맷팅

`datetime` 객체를 F-string으로 포맷팅할 수 있다.


In [ ]:
from datetime import datetime

now = datetime.now()
birthday = datetime(1995, 5, 15, 14, 30, 0)

print(f"현재 시간: {now}")
print(f"기본 포맷: {now:%Y-%m-%d %H:%M:%S}")
print(f"한국 날짜 형식: {now:%Y년%m월%d일}")
print(f"시간만: {now:%H:%M:%S}")
print(f"AM/PM 형식: {now:%Y-%m-%d %I:%M:%S %p}")
print(f"요일: {now:%A}")
print(f"월 이름: {now:%B}")
print(f"\n생일: {birthday:%Y년%m월%d일%H시%M분}")


## 문자열 정렬과 패딩

문자열을 정렬하고 특정 문자로 패딩할 수 있다.


In [ ]:
# 문자열 정렬과 패딩
text = "Python"
width = 20

print("기본 정렬:")
print(f"'{text:<{width}}'")
print(f"'{text:>{width}}'")
print(f"'{text:^{width}}'")

print("\n특수 문자로 패딩:")
print(f"'{text:-<{width}}'")
print(f"'{text:*>{width}}'")
print(f"'{text:=^{width}}'")

# 테이블 형태 출력
print("\n테이블 형태:")
data = [
    ("이름", "나이", "점수"),
    ("김철수", 25, 85),
    ("이영희", 30, 92),
    ("박민수", 22, 78)
]

for row in data:
    print(f"{row[0]:<8} {row[1]:>3} {row[2]:>4}")


## 실용 예제

진행률 표시와 로그 메시지 포맷팅


In [ ]:
# 진행률 표시
def show_progress(current, total):
    percentage = (current / total) * 100
    bar_length = 20
    filled_length = int(bar_length * current / total)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    return f"\r진행률: |{bar}| {percentage:.1f}% ({current}/{total})"

# 로그 메시지 포맷팅
def log_message(level, message):
    timestamp = datetime.now()
    return f"[{timestamp:%Y-%m-%d %H:%M:%S}] {level:>5}| {message}"

# 진행률 표시 예제
for i in range(0, 101, 20):
    print(show_progress(i, 100))

print("\n" + "="*50)

# 로그 메시지 예제
print(log_message("INFO", "프로그램이 시작되었습니다"))
print(log_message("ERROR", "파일을 찾을 수 없습니다"))
print(log_message("DEBUG", "변수 x의 값: 42"))
